In [1]:
'''
This file reads project/hand_pose/train_labels.csv and project/hand_pose/test_labels.csv and output to project/src/phase1_models/research/object_detection/data/train.record and project/src/phase1_models/research/object_detection/data/test.record
'''

'\nThis file reads project/hand_pose/train_labels.csv and project/hand_pose/test_labels.csv and output to project/src/phase1_models/research/object_detection/data/train.record and project/src/phase1_models/research/object_detection/data/test.record\n'

In [2]:
import sys
import os
import shutil
tensorflow_path = "phase1_models"
sys.path.append(tensorflow_path + "/research")
sys.path.append(tensorflow_path + "/research/object_detection")
sys.path.append(tensorflow_path + "/research/object_detection/utils")
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

import warnings
warnings.simplefilter("ignore")

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
base_dir = '../hand_pose'
output_dir = "phase1_models/research/object_detection/data"
csv_train_input= os.path.join(base_dir, 'train_labels.csv')
csv_test_input= os.path.join(base_dir, 'test_labels.csv')
csv_train_output=os.path.join(output_dir, 'train.record')
csv_test_output=os.path.join(output_dir, 'test.record')
img_path = os.path.join(base_dir, 'Color')
# shutil.move("../hand_pose/Gamma_imgs", "../hand_pose/Color")

In [4]:
def move(src, dst):
    listOfFiles = os.listdir(src)
    for f in listOfFiles:
        fullPath = src + "/" + f
        os.system ("mv"+ " " + fullPath + " " + dst)

In [5]:
move("../hand_pose/Gamma_imgs", "../hand_pose/Color")
move("../hand_pose/Rotate_270_imgs", "../hand_pose/Color")
move("../hand_pose/Rotate_90_imgs", "../hand_pose/Color")

In [6]:
# Before run following program, please copy all updated images from folders "Gamma_imgs" "Rotate_90_imgs" "Rotate_270_imgs" to target folder "Color" to get super dataset

In [7]:
# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == "hand":
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [8]:
writer = tf.python_io.TFRecordWriter(csv_train_output)
print("test:", csv_train_output,csv_train_input, writer)
examples = pd.read_csv(csv_train_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, img_path)
    writer.write(tf_example.SerializeToString())

writer.close()
print('Successfully created the TFRecords: {}'.format(csv_train_output))

test: phase1_models/research/object_detection/data/train.record ../hand_pose/train_labels.csv <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x125ee7cf8>
Successfully created the TFRecords: phase1_models/research/object_detection/data/train.record


In [9]:
writer = tf.python_io.TFRecordWriter(csv_test_output)
print("test:", csv_test_output,csv_test_input, writer)
examples = pd.read_csv(csv_test_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, img_path)
    writer.write(tf_example.SerializeToString())

writer.close()
print('Successfully created the TFRecords: {}'.format(csv_test_output))

test: phase1_models/research/object_detection/data/test.record ../hand_pose/test_labels.csv <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x125f835f8>
Successfully created the TFRecords: phase1_models/research/object_detection/data/test.record


In [10]:
# os.system("mv ../hand_pose/Color/*gamma.jpg ../hand_pose/Color/*r270.jpg ../hand_pose/Color/*r90.jpg")

In [11]:
import os

dir_name = "../hand_pose/Color/"
test = os.listdir(dir_name)

for item in test:
    if item.endswith("gamma.jpg") or item.endswith("r270.jpg") or item.endswith("r90.jpg"):
        os.remove(os.path.join(dir_name, item))